# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x2318cd08640>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15)) # The data

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
3047,3.2,1164.25,-200,20.327212,1305.50,259.0,648.25,198.0,1885.50,1217.5,35.525001,19.125,1.088833
4845,2.3,1049.75,-200,13.009989,1084.00,305.0,747.50,117.0,1402.00,1153.0,26.175000,24.425,0.817351
6049,6.6,1549.25,-200,30.101870,1553.25,1148.0,446.25,288.0,1795.50,2032.0,13.050000,55.050,0.823975
3881,1.1,1011.00,-200,6.085164,815.75,33.0,829.75,35.0,1613.75,680.0,36.850000,30.925,1.893422
8109,6.1,-200.00,-200,-200.000000,-200.00,1053.7,-200.00,312.4,-200.00,-200.0,-200.000000,-200.000,-200.000000


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :] # loc 通过标签或布尔数组访问一组行和列。iloc 按整数位置访问行和列组。

In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values 删除不要的features 竖列
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

In [7]:
np.random.seed(16166)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:
# Write your code here
## Clean the missing value
df = data_training_unproc
df.replace(to_replace = -200, value =np.nan,inplace=True) # We should not calcaluate the mean with -200 for na value
df.mean()

CO(GT)              2.154691
PT08.S1(CO)      1112.068888
C6H6(GT)           10.266127
PT08.S2(NMHC)     947.197536
NOx(GT)           257.652730
PT08.S3(NOx)      823.470866
NO2(GT)           115.355885
PT08.S4(NO2)     1441.616407
PT08.S5(O3)      1046.747827
T                  17.608795
RH                 49.215636
AH                  0.984057
dtype: float64

In [9]:
## Standardisation
clean_df = df.fillna(df.mean())
df_stand = (clean_df-clean_df.mean())/clean_df.std()

## 1.4 Training and validation stages

We have now curated our training data by
removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [10]:
# Write your code here
## Transform ytrain in to matrix
yTrain = np.matrix(np.array(df_stand.iloc[:,:1]))
## Trainsform XtR
XTrain =np.matrix(np.array(df_stand.iloc[:,1:]))

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [11]:
# Write your code here--allow-chromium-download
## Standarlize the validation data 
val = data_val_unproc.copy() # copy the data into new variable
val.replace(to_replace = -200, value =np.nan,inplace=True) # We should not calcaluate the mean with -200 for na value

In [12]:
clean_val = val.fillna(df.mean()) # replace the missing data with the mean in traning data
val_stand = (clean_val-clean_val.mean())/clean_val.std() # standarlisation

In [13]:
## Create a grid for parameters for 𝛾  and  𝜂  and S
gamma = np.logspace(-2 , 2.0, num = 5) # The regularistion parameter
eta = np.logspace(-2 , 2.0, num = 5) # Learning rate
S = np.linspace(20, 500,10,  dtype=int) # Size of minibatch

gamma, eta, S

(array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
 array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
 array([ 20,  73, 126, 180, 233, 286, 340, 393, 446, 500]))

In [14]:
## Objective function
def obj(X, y, w, lamda):
    inner = np.power(y - ((X * w.T)), 2) #RMSE
    fitting = np.sum(inner) / (X.shape[0]) # The first term in formula
    regul = (lamda/2)* np.sum(np.power(w, 2)) # The second term in formula
    return fitting + regul # final expression

In [15]:
## Make mini bacth randomly selected in XTrain set, the size is S we made before
def mini_batch(S):
    mini =df_stand.sample(S)
    m_y = np.matrix(np.array(mini.iloc[:,:1]))
    m_x = np.matrix(np.array(mini.iloc[:,1:]))
    return m_y, m_x

In [16]:
def batch_gradientDescent(X, y, w, S, learning_rate, lamda,iters):
    np.random.seed(16166)    
    sumterm = np.matrix(np.zeros((12,), dtype=int))
    for i in range(iters):
        m_y, m_x = mini_batch(S)
        for s in range(S):
            sumterm = 0 + ((m_y[s] - m_x[s] * w.T) * m_x[s])
            
        w_new = w * (1- learning_rate*lamda) + 2 * learning_rate/S * sumterm 
        w = w_new
        objective = obj(m_x, m_y, w_new, lamda)
    
        print("w: ", np.array(w), "Obj: ", objective)
        print("LR: ",learning_rate ,"S: ", S, "lamda: ",lamda,  i + 1)
    return w
                  

In [17]:
w = np.matrix(np.array([0]*XTrain.shape[1])) # W is a guessing value
# we can have a try to see the MSGD work or not
batch_gradientDescent(XTrain, yTrain, w, S[1], eta[1], gamma[1], 200) 

w:  [[ 7.59860886e-04  1.84107621e-04  3.04058977e-05 -2.85452459e-19
   1.56985078e-05 -3.17695984e-19  1.89915616e-04  8.49827949e-04
  -1.79186102e-03  1.83733128e-03  1.51291029e-04]] Obj:  0.9353940021678068
LR:  0.1 S:  73 lamda:  0.1 1
w:  [[ 0.03516496  0.05040051  0.03788201  0.03139868 -0.02062198  0.0090885
   0.03283087  0.03184841  0.0030741   0.00937392  0.01568234]] Obj:  0.8552423837186939
LR:  0.1 S:  73 lamda:  0.1 2
w:  [[ 0.03508113  0.05019735  0.03786931  0.03130732 -0.02084003  0.00906892
   0.03237066  0.03200862  0.0026898   0.00951975  0.01525988]] Obj:  0.6088028293513557
LR:  0.1 S:  73 lamda:  0.1 3
w:  [[ 0.04407632  0.06066615  0.04731827  0.04162517 -0.02781267  0.01400749
   0.04190529  0.03604116  0.00386315  0.01904537  0.02633512]] Obj:  0.4024880347613584
LR:  0.1 S:  73 lamda:  0.1 4
w:  [[ 0.05583574  0.06834607  0.05513202  0.04265513 -0.03236597  0.01708271
   0.05214094  0.04085825  0.00110485  0.0292241   0.02982608]] Obj:  0.6303685326611915


w:  [[ 0.08578791  0.10128951  0.09095685  0.0644855  -0.06092561  0.04894719
   0.06432098  0.07883121 -0.00531657  0.01298079  0.0044607 ]] Obj:  0.32554875919854626
LR:  0.1 S:  73 lamda:  0.1 78
w:  [[ 0.08649335  0.10147148  0.09143973  0.06507247 -0.06132605  0.05034445
   0.06342941  0.07916928 -0.00600882  0.01203143  0.00222343]] Obj:  0.28471446381440607
LR:  0.1 S:  73 lamda:  0.1 79
w:  [[ 0.08577767  0.10060549  0.09069971  0.06455115 -0.06100153  0.05001488
   0.0628856   0.0786024  -0.00593307  0.01199739  0.00229312]] Obj:  0.18117762563690185
LR:  0.1 S:  73 lamda:  0.1 80
w:  [[ 0.08687233  0.10330503  0.09321773  0.06431373 -0.06224681  0.05310522
   0.06578857  0.08011875 -0.00149873  0.00855862  0.00355188]] Obj:  0.23699778328654547
LR:  0.1 S:  73 lamda:  0.1 81
w:  [[ 0.08625537  0.10240503  0.09240035  0.06362302 -0.06170506  0.05260095
   0.06545496  0.07946071 -0.00146855  0.00882813  0.00383638]] Obj:  0.37678629798482266
LR:  0.1 S:  73 lamda:  0.1 82
w:  [

w:  [[ 0.10025476  0.12420673  0.11092981  0.08396551 -0.06661489  0.0560323
   0.07269342  0.09398226 -0.00895847  0.01129785 -0.00030517]] Obj:  0.3196059544387431
LR:  0.1 S:  73 lamda:  0.1 144
w:  [[ 0.101105    0.12481877  0.11156072  0.08453745 -0.06761105  0.05784174
   0.0726732   0.09522011 -0.00876847  0.010552   -0.00074868]] Obj:  0.15959256384934206
LR:  0.1 S:  73 lamda:  0.1 145
w:  [[ 0.09946272  0.12312193  0.11003401  0.08388292 -0.06273116  0.05718397
   0.07104663  0.09387037 -0.00911246  0.00989668 -0.00158983]] Obj:  0.1570521298317593
LR:  0.1 S:  73 lamda:  0.1 146
w:  [[ 0.09850428  0.12200963  0.10910011  0.08311917 -0.06210381  0.05690776
   0.0703694   0.09299872 -0.00864968  0.00943927 -0.00148004]] Obj:  0.1499419858793813
LR:  0.1 S:  73 lamda:  0.1 147
w:  [[ 0.09880032  0.12190747  0.10923094  0.08335006 -0.06314311  0.05777618
   0.07031993  0.09331363 -0.00939326  0.01004835 -0.00190274]] Obj:  0.39974440373324777
LR:  0.1 S:  73 lamda:  0.1 148
w:  

matrix([[ 0.10783604,  0.12858145,  0.1154246 ,  0.08420657, -0.07244113,
          0.06608292,  0.07930238,  0.10037795,  0.00320548, -0.00489986,
          0.00188046]])

In [18]:
# grid search
# firstly should did learning rate
for i in range(5):   
     batch_gradientDescent(XTrain, yTrain, w, S[1], eta[i], gamma[1], 200)

w:  [[ 7.59860886e-05  1.84107621e-05  3.04058977e-06 -2.85452459e-20
   1.56985078e-06 -3.17695984e-20  1.89915616e-05  8.49827949e-05
  -1.79186102e-04  1.83733128e-04  1.51291029e-05]] Obj:  0.9374490709137275
LR:  0.01 S:  73 lamda:  0.1 1
w:  [[ 0.00352245  0.00504791  0.00379403  0.00314468 -0.00206535  0.00091024
   0.00328826  0.00319036  0.00030654  0.0009402   0.00157075]] Obj:  1.261126251077662
LR:  0.01 S:  73 lamda:  0.1 2
w:  [[ 0.00356191  0.00509115  0.003849    0.00317727 -0.00213137  0.00092078
   0.0032638   0.00326399  0.00024949  0.00097771  0.00152655]] Obj:  0.92438943076266
LR:  0.01 S:  73 lamda:  0.1 3
w:  [[ 0.00472581  0.00645647  0.00507276  0.00450205 -0.00302626  0.00154808
   0.00449199  0.00380444  0.00039917  0.00217852  0.00292754]] Obj:  0.7561793896070662
LR:  0.01 S:  73 lamda:  0.1 4
w:  [[ 6.14557818e-03  7.41755678e-03  6.03527704e-03  4.66640443e-03
  -3.58735237e-03  1.92195475e-03  5.73154597e-03  4.40516072e-03
   8.12199718e-05  3.38704798

w:  [[ 0.01541854  0.01706142  0.01558586  0.01197188 -0.01160861  0.00959065
   0.01147647  0.01402255 -0.00041632  0.00424323  0.0030764 ]] Obj:  1.337119113630018
LR:  0.01 S:  73 lamda:  0.1 64
w:  [[ 0.01542789  0.01707751  0.01561339  0.0119812  -0.01165707  0.0095102
   0.01151776  0.01403275 -0.00039726  0.0043355   0.00319663]] Obj:  0.9415866214805317
LR:  0.01 S:  73 lamda:  0.1 65
w:  [[ 0.01547634  0.01717919  0.01571821  0.01200089 -0.01166325  0.00957018
   0.01166526  0.01406499 -0.00023627  0.00416899  0.00328539]] Obj:  1.3844391345917297
LR:  0.01 S:  73 lamda:  0.1 66
w:  [[ 0.01610172  0.01797938  0.01643735  0.01265905 -0.01220803  0.00961371
   0.01225427  0.01462201 -0.00010172  0.00446896  0.00379849]] Obj:  0.7629286492395461
LR:  0.01 S:  73 lamda:  0.1 67
w:  [[ 0.0161495   0.01803562  0.01648882  0.01275226 -0.01235165  0.00971071
   0.01229402  0.01474215 -0.00014997  0.00464147  0.00391536]] Obj:  0.6810188474255239
LR:  0.01 S:  73 lamda:  0.1 68
w:  [[ 

w:  [[ 0.03257864  0.03800619  0.0351234   0.02449957 -0.02722222  0.01861048
   0.02538577  0.03023546  0.00085908  0.00534024  0.00518852]] Obj:  0.5537775187928893
LR:  0.01 S:  73 lamda:  0.1 135
w:  [[ 0.03277829  0.03819651  0.03532418  0.02475288 -0.02742464  0.01889929
   0.02550087  0.03059878  0.00022359  0.00587869  0.00519112]] Obj:  0.5421489162742751
LR:  0.01 S:  73 lamda:  0.1 136
w:  [[ 0.03273623  0.03815939  0.03528499  0.02471926 -0.02737009  0.01881529
   0.02549016  0.03056051  0.00025757  0.00584448  0.00520847]] Obj:  0.8546411426718862
LR:  0.01 S:  73 lamda:  0.1 137
w:  [[ 0.03274337  0.03816673  0.03529001  0.02475255 -0.02743393  0.01884881
   0.02548632  0.03061136  0.00024969  0.00593451  0.00528116]] Obj:  0.6591768501124405
LR:  0.01 S:  73 lamda:  0.1 138
w:  [[ 0.03290861  0.03831482  0.03546152  0.02482861 -0.02753411  0.01885286
   0.02580543  0.03082119  0.00042353  0.00613387  0.00557464]] Obj:  0.5385826736291872
LR:  0.01 S:  73 lamda:  0.1 139


w:  [[ 7.59860886e-04  1.84107621e-04  3.04058977e-05 -2.85452459e-19
   1.56985078e-05 -3.17695984e-19  1.89915616e-04  8.49827949e-04
  -1.79186102e-03  1.83733128e-03  1.51291029e-04]] Obj:  0.9353940021678068
LR:  0.1 S:  73 lamda:  0.1 1
w:  [[ 0.03516496  0.05040051  0.03788201  0.03139868 -0.02062198  0.0090885
   0.03283087  0.03184841  0.0030741   0.00937392  0.01568234]] Obj:  0.8552423837186939
LR:  0.1 S:  73 lamda:  0.1 2
w:  [[ 0.03508113  0.05019735  0.03786931  0.03130732 -0.02084003  0.00906892
   0.03237066  0.03200862  0.0026898   0.00951975  0.01525988]] Obj:  0.6088028293513557
LR:  0.1 S:  73 lamda:  0.1 3
w:  [[ 0.04407632  0.06066615  0.04731827  0.04162517 -0.02781267  0.01400749
   0.04190529  0.03604116  0.00386315  0.01904537  0.02633512]] Obj:  0.4024880347613584
LR:  0.1 S:  73 lamda:  0.1 4
w:  [[ 0.05583574  0.06834607  0.05513202  0.04265513 -0.03236597  0.01708271
   0.05214094  0.04085825  0.00110485  0.0292241   0.02982608]] Obj:  0.6303685326611915


w:  [[ 0.08314019  0.09408677  0.08497264  0.06545017 -0.0613459   0.04826094
   0.057804    0.07445101 -0.00435658  0.01756403  0.01033001]] Obj:  0.270692699465536
LR:  0.1 S:  73 lamda:  0.1 72
w:  [[ 0.08154389  0.09478463  0.08545192  0.06408907 -0.05890508  0.04857652
   0.05823475  0.07387059 -0.00230182  0.01276468  0.00971692]] Obj:  0.2883128303038181
LR:  0.1 S:  73 lamda:  0.1 73
w:  [[ 0.08060916  0.093767    0.08450852  0.06334195 -0.05819449  0.04813284
   0.05753439  0.07312288 -0.00231413  0.0123796   0.00931262]] Obj:  0.4598911079643508
LR:  0.1 S:  73 lamda:  0.1 74
w:  [[ 0.08104355  0.09423771  0.08517176  0.06428067 -0.05818882  0.04976961
   0.05650946  0.07467703 -0.00381946  0.01113266  0.0049628 ]] Obj:  0.4386285061118039
LR:  0.1 S:  73 lamda:  0.1 75
w:  [[ 0.08029556  0.0932861   0.08434548  0.06366152 -0.05775608  0.04956548
   0.05576108  0.07393055 -0.00402062  0.01096828  0.00463458]] Obj:  0.39281668077059606
LR:  0.1 S:  73 lamda:  0.1 76
w:  [[ 0.0

...

LR:  100.0 S:  73 lamda:  0.1 33
w:  [[-1.90486847e+41 -1.66946879e+41 -2.10340237e+41 -2.77881709e+39
   1.75694639e+41 -1.11446785e+40 -1.81365733e+41 -1.42809950e+41
  -1.05941354e+41  1.01775951e+40 -7.71464170e+40]] Obj:  1.1356192881237225e+84
LR:  100.0 S:  73 lamda:  0.1 34
w:  [[ 2.10188206e+42  2.00358425e+42  2.19926935e+42  1.18814478e+42
  -2.63575371e+42  7.60511740e+41  2.00540032e+42  3.00072986e+42
   1.06083240e+42  1.72545755e+42  2.36783898e+42]] Obj:  2.1800718069763056e+86
LR:  100.0 S:  73 lamda:  0.1 35
w:  [[-1.92243188e+43 -1.90177713e+43 -2.10178785e+43 -1.19902200e+43
   2.51565283e+43 -9.56913677e+42 -1.64881987e+43 -2.86245560e+43
  -7.74661730e+42 -1.31924844e+43 -1.82163641e+43]] Obj:  1.9468741760619674e+88
LR:  100.0 S:  73 lamda:  0.1 36
w:  [[ 2.07104688e+44  1.98089314e+44  2.08657654e+44  1.11868135e+44
  -2.21687919e+44  1.25592515e+44  1.82205951e+44  2.53553539e+44
   7.73545687e+43  8.30494712e+43  1.45115823e+44]] Obj:  1.3654259762194192e+90


w:  [[ 8.19615355e+128  8.07308065e+128  1.15763037e+129  5.85107669e+128
  -1.70089387e+129  1.07795834e+129  1.18938637e+129  1.04196899e+129
   1.07506380e+129 -2.37795444e+128  7.47837339e+128]] Obj:  6.936265829828365e+259
LR:  100.0 S:  73 lamda:  0.1 102
w:  [[-1.64410054e+130 -1.41701919e+130 -1.68989016e+130 -8.45298926e+129
   1.66857025e+130 -1.82067294e+130 -1.29171632e+130 -1.50992346e+130
  -1.07434383e+130  2.03299578e+130  6.78198182e+129]] Obj:  1.1218375637689292e+262
LR:  100.0 S:  73 lamda:  0.1 103
w:  [[ 6.10913814e+131  6.22670110e+131  7.48543794e+131  5.20111067e+131
  -1.15975070e+132  6.99544256e+131  4.81391925e+131  8.37052776e+131
   4.51342396e+131 -1.50331915e+131  3.55891533e+131]] Obj:  2.124872781957779e+265
LR:  100.0 S:  73 lamda:  0.1 104
w:  [[-1.58488944e+133 -1.50469052e+133 -1.79238511e+133 -2.52796372e+133
   2.74350967e+133 -4.87903079e+133 -4.04676725e+132 -1.92646865e+133
   1.49388292e+133 -1.94501775e+133  7.12037722e+132]] Obj:  2.456546

w:  [[ 1.94966430e+214  3.35878521e+214  3.13713013e+214  3.82648811e+214
  -2.85264166e+214  3.55964985e+214  2.04243320e+214  3.43774678e+214
  -2.52008483e+214  2.19200702e+214 -6.30567787e+212]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 165
w:  [[-1.45515229e+215 -2.55818465e+215 -2.51158348e+215 -3.65487838e+215
   2.94792405e+215 -3.54531510e+215 -4.46461701e+214 -3.49360059e+215
   4.08766439e+215 -3.17626436e+215  1.35808965e+215]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 166
w:  [[ 1.44537320e+216  2.57632687e+216  2.81208824e+216  4.08168577e+216
  -3.79765638e+216  3.58871221e+216  5.96784804e+215  3.15762569e+216
  -3.66544054e+216  3.50990930e+216 -6.42883207e+215]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 167
w:  [[-4.74077225e+217 -7.33959482e+217 -8.14926973e+217 -8.42199439e+217
   1.06608961e+218 -8.42790523e+217 -2.31399713e+217 -1.03254505e+218
   1.22933592e+216  2.06765151e+217  7.91114343e+215]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 168
w:  [[ 1.45273297e+219  7.91

C:\Users\ZHOUQI~1\AppData\Local\Temp/ipykernel_19584/2502419380.py:3: RuntimeWarning: overflow encountered in power
  inner = np.power(y - ((X * w.T)), 2) #RMSE
E:\langurages\python3.9\lib\site-packages\numpy\core\_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\ZHOUQI~1\AppData\Local\Temp/ipykernel_19584/2502419380.py:5: RuntimeWarning: overflow encountered in power
  regul = (lamda/2)* np.sum(np.power(w, 2)) # The second term in formula


 [[ 8.30914844e+228  7.44121402e+228  7.64163810e+228  7.25288673e+228
  -1.22715390e+229  7.98290799e+228  4.55666761e+228  1.18737738e+229
   1.82512045e+228  6.58477087e+228  7.58888058e+228]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 177
w:  [[-2.61192297e+230 -2.80657027e+230 -3.21110003e+230 -2.52747357e+230
   4.35050028e+230 -3.47845020e+230 -1.09337107e+230 -2.34470168e+230
  -1.83414859e+230  1.40099079e+230 -1.45555125e+230]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 178
w:  [[ 2.41013299e+231  2.71006604e+231  3.17733096e+231  1.83883837e+231
  -3.91537217e+231  2.78474681e+231  1.41542531e+231  1.91336517e+231
   2.33915489e+231 -2.12506186e+231  1.21443362e+231]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 179
w:  [[-1.69156840e+232 -2.57799775e+232 -3.20794986e+232 -9.30409170e+231
   3.87940721e+232 -2.74945872e+232 -2.07949228e+232 -1.36303538e+232
  -3.10965468e+232  2.94495005e+232 -1.37876221e+232]] Obj:  inf
LR:  100.0 S:  73 lamda:  0.1 180
w:  [[ 8.54259926e+233  8.95142

In [19]:
# For learning rate s = 0.1 is best 
for i in range(5):   
     batch_gradientDescent(XTrain, yTrain, w, S[i], eta[1], gamma[1], 200)

w:  [[ 0.00643977  0.00673664  0.00696254  0.00354669 -0.00315053 -0.00094174
   0.01305254  0.00817902  0.00762773  0.00721832  0.01199153]] Obj:  0.640986735840917
LR:  0.1 S:  20 lamda:  0.1 1
w:  [[ 0.00652318  0.00668828  0.00696567  0.00371873 -0.00340852 -0.0002351
   0.01278086  0.00824115  0.00719402  0.00748701  0.01165772]] Obj:  0.9210140244677308
LR:  0.1 S:  20 lamda:  0.1 2
w:  [[ 0.01305789  0.01083498  0.01255811  0.01430489 -0.01329482  0.01397215
   0.00932239  0.01782831 -0.00068129  0.00676046  0.00235173]] Obj:  0.5072837547256013
LR:  0.1 S:  20 lamda:  0.1 3
w:  [[ 0.02374318  0.02239968  0.02489977  0.04305085 -0.02642437  0.01521943
   0.00996464  0.02898461 -0.01464917  0.03037949 -0.00147091]] Obj:  0.8648433914388092
LR:  0.1 S:  20 lamda:  0.1 4
w:  [[ 0.02889439  0.02530947  0.02713811  0.04669097 -0.02548031  0.02329997
   0.01101956  0.03034473 -0.01662653  0.02553395 -0.00923375]] Obj:  0.597473680106385
LR:  0.1 S:  20 lamda:  0.1 5
w:  [[ 0.03007148 

w:  [[ 0.1167144   0.13626165  0.12823413  0.10323967 -0.08368474  0.06974266
   0.08348214  0.08681617  0.00233215 -0.00652211 -0.03216256]] Obj:  0.12375083311811937
LR:  0.1 S:  20 lamda:  0.1 63
w:  [[ 0.10360872  0.12481581  0.11670803  0.09299952 -0.07102695  0.06273523
   0.0761331   0.07551655  0.00558172 -0.01430385 -0.03325843]] Obj:  0.12181487572684949
LR:  0.1 S:  20 lamda:  0.1 64
w:  [[ 0.10511148  0.12637872  0.11834708  0.09586428 -0.07086977  0.06803039
   0.07366102  0.07782015  0.00159565 -0.01526204 -0.04153009]] Obj:  0.10323874241195821
LR:  0.1 S:  20 lamda:  0.1 65
w:  [[ 0.10710745  0.12622178  0.11773419  0.09845113 -0.07003387  0.07059357
   0.07164784  0.07736213 -0.00053129 -0.01365815 -0.04266608]] Obj:  1.0844226577410756
LR:  0.1 S:  20 lamda:  0.1 66
w:  [[ 0.10602148  0.12548596  0.11677357  0.09585596 -0.06813388  0.06474256
   0.07363574  0.07559854  0.00176817 -0.01183553 -0.0389673 ]] Obj:  0.2566149742368675
LR:  0.1 S:  20 lamda:  0.1 67
w:  [[ 

w:  [[ 0.14635511  0.16992116  0.1488869   0.12022695 -0.06250977  0.08312987
   0.11418086  0.12017503  0.00426151 -0.00659291 -0.01314611]] Obj:  0.044672736222406334
LR:  0.1 S:  20 lamda:  0.1 133
w:  [[ 0.14046913  0.16480303  0.14291588  0.11592668 -0.05414091  0.07736585
   0.11199884  0.1144263   0.00264503 -0.00285553 -0.01255372]] Obj:  0.14023166597522171
LR:  0.1 S:  20 lamda:  0.1 134
w:  [[ 0.13704179  0.16336637  0.14145308  0.11580279 -0.05409807  0.077369
   0.10995393  0.1128407   0.0023463  -0.00222584 -0.01223128]] Obj:  0.14720761524791853
LR:  0.1 S:  20 lamda:  0.1 135
w:  [[ 0.15033217  0.18633889  0.15936705  0.12889759 -0.06468293  0.0769543
   0.12389685  0.12258503  0.00150431  0.00530669 -0.00710906]] Obj:  0.2731212946397087
LR:  0.1 S:  20 lamda:  0.1 136
w:  [[ 0.14878459  0.18462189  0.15813066  0.12687259 -0.06421907  0.07569303
   0.12379659  0.12096555  0.00158034  0.00528369 -0.00689196]] Obj:  0.16571555393634918
LR:  0.1 S:  20 lamda:  0.1 137
w: 

w:  [[ 0.15171434  0.17456271  0.14109409  0.12397432 -0.02463318  0.10937654
   0.11832658  0.10121396 -0.0059008  -0.02776861 -0.03382102]] Obj:  0.07345073597219753
LR:  0.1 S:  20 lamda:  0.1 196
w:  [[ 0.15018395  0.17284081  0.13967515  0.12287061 -0.02427777  0.10848078
   0.11689979  0.10020888 -0.00612678 -0.02753881 -0.03410248]] Obj:  0.10306620398879882
LR:  0.1 S:  20 lamda:  0.1 197
w:  [[ 0.14976252  0.17162165  0.13862069  0.12246916 -0.02411121  0.10802191
   0.11524524  0.09931951 -0.00642268 -0.02714076 -0.03414614]] Obj:  0.05533203491505065
LR:  0.1 S:  20 lamda:  0.1 198
w:  [[ 0.14817486  0.16989263  0.13725182  0.12109229 -0.02386791  0.10684857
   0.11423086  0.09818536 -0.00597081 -0.02717574 -0.03368141]] Obj:  0.1040378450377315
LR:  0.1 S:  20 lamda:  0.1 199
w:  [[ 0.14845639  0.16960663  0.13745418  0.12143307 -0.02486462  0.10807678
   0.11256176  0.09836663 -0.00711109 -0.0273592  -0.0359398 ]] Obj:  0.13291302278516617
LR:  0.1 S:  20 lamda:  0.1 200
w

w:  [[ 0.08330797  0.09502247  0.08483855  0.06684381 -0.05834875  0.05061244
   0.06013976  0.07455379 -0.00487974  0.01901527  0.01039763]] Obj:  0.2781890707065173
LR:  0.1 S:  73 lamda:  0.1 67
w:  [[ 0.08251242  0.09411584  0.08403005  0.06623756 -0.05785679  0.05016894
   0.05956891  0.0738874  -0.00485934  0.01892907  0.01036453]] Obj:  0.2686196978764603
LR:  0.1 S:  73 lamda:  0.1 68
w:  [[ 0.08122896  0.09302825  0.08292166  0.06610931 -0.05724419  0.05006761
   0.05839684  0.07328379 -0.00552723  0.01949846  0.01012433]] Obj:  0.328865245217243
LR:  0.1 S:  73 lamda:  0.1 69
w:  [[ 0.08366905  0.09476258  0.08545978  0.06544821 -0.06113492  0.04956694
   0.05859805  0.07481612 -0.00422991  0.0171247   0.01010818]] Obj:  0.2659223581371528
LR:  0.1 S:  73 lamda:  0.1 70
w:  [[ 0.08356954  0.09426769  0.08496897  0.06558419 -0.06110036  0.04952875
   0.05754827  0.07440025 -0.00462334  0.01686911  0.00925693]] Obj:  0.31071593438034506
LR:  0.1 S:  73 lamda:  0.1 71
w:  [[ 0.0

...

w:  [[ 0.06545468  0.07226233  0.06772959  0.05531051 -0.04501057  0.04527673
   0.04792461  0.05716545  0.00386631  0.00246501  0.00515074]] Obj:  0.3291585196352783
LR:  0.1 S:  180 lamda:  0.1 185
w:  [[ 0.06480038  0.07153976  0.06705215  0.05475816 -0.04456039  0.04482416
   0.04744448  0.05659451  0.0038253   0.00244208  0.00509811]] Obj:  0.5626804221242158
LR:  0.1 S:  180 lamda:  0.1 186
w:  [[ 0.0641861   0.07076558  0.06628726  0.05437438 -0.04406585  0.0444215
   0.04674904  0.05612401  0.00320574  0.00282451  0.00483538]] Obj:  0.36752418298271755
LR:  0.1 S:  180 lamda:  0.1 187
w:  [[ 0.06354424  0.07005793  0.06562439  0.05383064 -0.04362519  0.04397729
   0.04628155  0.05556277  0.00317368  0.00279627  0.00478703]] Obj:  0.5663142472985201
LR:  0.1 S:  180 lamda:  0.1 188
w:  [[ 0.06282923  0.06935037  0.06494406  0.05329096 -0.04307155  0.04354432
   0.045812    0.05498257  0.00305851  0.00283672  0.00469306]] Obj:  0.4047385330640582
LR:  0.1 S:  180 lamda:  0.1 189


w:  [[ 1.67004952e-02  1.92480542e-02  1.79933953e-02  8.08501420e-03
  -1.13846512e-02  9.18464603e-03  1.38925948e-02  1.48419691e-02
   3.26958600e-03 -4.75206491e-03 -5.84303018e-05]] Obj:  0.7795475409680587
LR:  0.1 S:  233 lamda:  0.1 27
w:  [[ 0.01731878  0.01988985  0.01881192  0.00879917 -0.01272284  0.01019527
   0.01412458  0.01581487  0.00340835 -0.00451623  0.00031006]] Obj:  0.8799821444563318
LR:  0.1 S:  233 lamda:  0.1 28
w:  [[ 0.0177187   0.02030332  0.01933392  0.00912481 -0.01297716  0.0104479
   0.01486832  0.01609012  0.00430522 -0.00503544  0.00097999]] Obj:  0.9575648083064711
LR:  0.1 S:  233 lamda:  0.1 29
w:  [[ 0.01793082  0.02054016  0.01962163  0.00971109 -0.01328148  0.0108757
   0.01457302  0.01652101  0.00349558 -0.00475706  0.00033067]] Obj:  0.8136181068180757
LR:  0.1 S:  233 lamda:  0.1 30
w:  [[ 0.01839241  0.02116935  0.02024296  0.01011798 -0.01387073  0.01066687
   0.01501306  0.01699675  0.00366842 -0.00447605  0.00087902]] Obj:  0.7836588025

w:  [[ 0.02837029  0.02896874  0.029567    0.01734264 -0.02089424  0.01520827
   0.0235414   0.02731446  0.00699975 -0.0011179   0.00585074]] Obj:  0.5830145213714847
LR:  0.1 S:  233 lamda:  0.1 70
w:  [[ 0.02875138  0.02928575  0.02989967  0.01716922 -0.02087793  0.01505619
   0.02357019  0.02746037  0.00699465 -0.00210149  0.00505646]] Obj:  0.6423346219264937
LR:  0.1 S:  233 lamda:  0.1 71
w:  [[ 0.02873667  0.02934442  0.02995717  0.01737675 -0.02124526  0.0151215
   0.0234838   0.02749247  0.00731085 -0.00234357  0.00529151]] Obj:  0.7488603808698328
LR:  0.1 S:  233 lamda:  0.1 72
w:  [[ 0.02909851  0.02972194  0.03042171  0.01789208 -0.02177084  0.01587894
   0.02324157  0.028144    0.00636718 -0.00199708  0.00417284]] Obj:  0.6727093315348698
LR:  0.1 S:  233 lamda:  0.1 73
w:  [[ 0.02920339  0.0298938   0.03059769  0.01775292 -0.02192133  0.0162559
   0.02349748  0.02808727  0.00661439 -0.00247767  0.00417984]] Obj:  0.8539740359077632
LR:  0.1 S:  233 lamda:  0.1 74
w:  [[ 

w:  [[ 0.03875196  0.03736593  0.03648076  0.03662021 -0.02591686  0.03718546
   0.02441368  0.03720435  0.00058208 -0.00322883 -0.00307804]] Obj:  0.5954175092423004
LR:  0.1 S:  233 lamda:  0.1 114
w:  [[ 0.0383857   0.03697093  0.03608616  0.03628262 -0.02564689  0.03680077
   0.02411515  0.03685118  0.00048945 -0.00312141 -0.00307398]] Obj:  0.5101407024291251
LR:  0.1 S:  233 lamda:  0.1 115
w:  [[ 0.03802021  0.03657642  0.03569014  0.03595962 -0.02537407  0.03644357
   0.02382851  0.03644324  0.00039516 -0.00299247 -0.0030424 ]] Obj:  0.7194022266286351
LR:  0.1 S:  233 lamda:  0.1 116
w:  [[ 0.03798722  0.03655638  0.03570908  0.03560002 -0.02528375  0.03607914
   0.02400981  0.03623857  0.00082287 -0.00339645 -0.00276286]] Obj:  0.5579403908128854
LR:  0.1 S:  233 lamda:  0.1 117
w:  [[ 0.03795586  0.03636795  0.03554552  0.03516493 -0.02503205  0.03593886
   0.02395236  0.03602351  0.00102734 -0.00365862 -0.00281747]] Obj:  0.6296463200784305
LR:  0.1 S:  233 lamda:  0.1 118


w:  [[ 0.03679168  0.03967768  0.03975954  0.0356391  -0.0281347   0.03712346
   0.02667745  0.03715324  0.00087916 -0.0049983  -0.00167338]] Obj:  0.519923421163123
LR:  0.1 S:  233 lamda:  0.1 157
w:  [[ 3.77874360e-02  4.04227627e-02  4.06198398e-02  3.71978233e-02
  -2.94195013e-02  3.95409707e-02  2.62582691e-02  3.84520877e-02
   2.62002726e-05 -5.47031369e-03 -2.95856591e-03]] Obj:  0.8036532609246302
LR:  0.1 S:  233 lamda:  0.1 158
w:  [[ 0.03752961  0.04023278  0.04041859  0.03697686 -0.02919051  0.03939838
   0.02621301  0.03820259  0.00025416 -0.00575145 -0.00286233]] Obj:  0.9338305296383246
LR:  0.1 S:  233 lamda:  0.1 159
w:  [[ 0.03766753  0.04039057  0.04058307  0.03691512 -0.02936192  0.03907652
   0.026532    0.03825094  0.00066667 -0.00573147 -0.00218498]] Obj:  0.5732023786116002
LR:  0.1 S:  233 lamda:  0.1 160
w:  [[ 0.03770373  0.04039809  0.04065961  0.03690393 -0.02986705  0.03916675
   0.026517    0.03849033  0.00070335 -0.00543551 -0.00190882]] Obj:  0.63274

In [20]:
# S = 20 is the best value
for i in range(5):   
     batch_gradientDescent(XTrain, yTrain, w, S[0], eta[1], gamma[i], 200)

w:  [[ 0.00643977  0.00673664  0.00696254  0.00354669 -0.00315053 -0.00094174
   0.01305254  0.00817902  0.00762773  0.00721832  0.01199153]] Obj:  0.640957482682971
LR:  0.1 S:  20 lamda:  0.01 1
w:  [[ 0.00658114  0.00674891  0.00702833  0.00375065 -0.00343688 -0.00024357
   0.01289834  0.00831476  0.00726267  0.00755198  0.01176564]] Obj:  0.9203073569882795
LR:  0.1 S:  20 lamda:  0.01 2
w:  [[ 0.01317433  0.01095564  0.01268326  0.01436999 -0.01335358  0.01396122
   0.00955486  0.01797578 -0.00054778  0.00689276  0.00256469]] Obj:  0.5065320315673546
LR:  0.1 S:  20 lamda:  0.01 3
w:  [[ 0.02396963  0.02260975  0.02512929  0.04322486 -0.02659364  0.01533331
   0.01028027  0.02928463 -0.01451235  0.0305563  -0.00123439]] Obj:  0.8613563799553101
LR:  0.1 S:  20 lamda:  0.01 4
w:  [[ 0.02933189  0.02571953  0.02759039  0.04725044 -0.02588754  0.02354704
   0.01142404  0.03090458 -0.01662075  0.02598602 -0.00900724]] Obj:  0.5936641571562179
LR:  0.1 S:  20 lamda:  0.01 5
w:  [[ 0.03

w:  [[ 0.12106173  0.14551108  0.13556723  0.12818737 -0.08050263  0.07972038
   0.0805917   0.07933206 -0.00060429 -0.00773979 -0.04778875]] Obj:  0.8742494076140795
LR:  0.1 S:  20 lamda:  0.01 66
w:  [[ 0.12092341  0.14597569  0.13568285  0.12619236 -0.07903168  0.07367732
   0.08364559  0.07810532  0.00205533 -0.00577787 -0.04394845]] Obj:  0.17643983927445486
LR:  0.1 S:  20 lamda:  0.01 67
w:  [[ 0.1224209   0.14857813  0.13836477  0.12907157 -0.08175228  0.07715613
   0.08309754  0.08014528  0.00115523 -0.0056473  -0.04505887]] Obj:  0.09482066993665787
LR:  0.1 S:  20 lamda:  0.01 68
w:  [[ 0.14028574  0.16630783  0.15330684  0.14088815 -0.09329439  0.08824439
   0.09351627  0.09429867  0.00294932 -0.00784305 -0.04467747]] Obj:  0.0498972908758591
LR:  0.1 S:  20 lamda:  0.01 69
w:  [[ 0.13566687  0.16175435  0.1481558   0.13816235 -0.0722211   0.08403515
   0.08995445  0.09091302  0.00017549 -0.00077086 -0.04355551]] Obj:  0.09152444174959014
LR:  0.1 S:  20 lamda:  0.01 70
w:

w:  [[ 0.16278059  0.19755241  0.17270621  0.14852839 -0.05956925  0.08710668
   0.12561937  0.11288051  0.00201526  0.00295466 -0.02983536]] Obj:  0.2555302234755338
LR:  0.1 S:  20 lamda:  0.01 131
w:  [[ 0.1635898   0.19877877  0.17370804  0.14964502 -0.06023112  0.08736196
   0.12645029  0.11379577  0.00224651  0.00314821 -0.0292256 ]] Obj:  0.05290756564202416
LR:  0.1 S:  20 lamda:  0.01 132
w:  [[ 0.15847465  0.19525542  0.16866856  0.14949538 -0.05118522  0.0872746
   0.12183146  0.10907693  0.00038969  0.0014499  -0.03226969]] Obj:  0.05261831473459097
LR:  0.1 S:  20 lamda:  0.01 133
w:  [[ 0.15280223  0.19079742  0.16291149  0.14548326 -0.04147902  0.08103715
   0.12041267  0.10329862 -0.00157301  0.00602607 -0.03166273]] Obj:  0.10537459968633503
LR:  0.1 S:  20 lamda:  0.01 134
w:  [[ 0.15067862  0.19081257  0.16271579  0.14634661 -0.04192333  0.081713
   0.11939106  0.1027653  -0.00183674  0.00660576 -0.03143921]] Obj:  0.1343519554406033
LR:  0.1 S:  20 lamda:  0.01 135


LR:  0.1 S:  20 lamda:  0.01 192
w:  [[ 0.16989189  0.21514893  0.16679612  0.16378802  0.0185526   0.11098194
   0.13659454  0.08132835 -0.0063135  -0.02549869 -0.05123293]] Obj:  0.2213869253572807
LR:  0.1 S:  20 lamda:  0.01 193
w:  [[ 0.16929336  0.21802682  0.17021994  0.16263872  0.01607594  0.11203509
   0.14092957  0.08241092 -0.00329548 -0.02890263 -0.05069197]] Obj:  0.28473361962075183
LR:  0.1 S:  20 lamda:  0.01 194
w:  [[ 0.1701671   0.21886182  0.17123832  0.16247608  0.01516503  0.11192305
   0.14135925  0.08382631 -0.00502168 -0.02749793 -0.05134583]] Obj:  0.20654536584221653
LR:  0.1 S:  20 lamda:  0.01 195
w:  [[ 0.16954728  0.21825745  0.17073234  0.16220552  0.01541844  0.11160586
   0.14027319  0.08353573 -0.00599885 -0.02748586 -0.05225073]] Obj:  0.06159690253044544
LR:  0.1 S:  20 lamda:  0.01 196
w:  [[ 0.16941402  0.21797421  0.17058349  0.16167079  0.0151043   0.11095199
   0.14079984  0.08343288 -0.00521237 -0.02732725 -0.05050143]] Obj:  0.06600123857661

w:  [[ 0.11894837  0.14687479  0.13816721  0.11378045 -0.08730145  0.07351681
   0.0923531   0.09063438  0.00694525 -0.00303029 -0.02626634]] Obj:  0.0963917408877419
LR:  0.1 S:  20 lamda:  0.1 52
w:  [[ 0.11578267  0.14390876  0.13584042  0.11387853 -0.08758168  0.07513782
   0.087187    0.08859268  0.0026211  -0.00286227 -0.03053886]] Obj:  0.2705728145712155
LR:  0.1 S:  20 lamda:  0.1 53
w:  [[ 0.11312415  0.13692504  0.1283562   0.10887384 -0.08796234  0.07092426
   0.0820202   0.0845581   0.00115021  0.0014611  -0.02883762]] Obj:  0.10868365379228782
LR:  0.1 S:  20 lamda:  0.1 54
w:  [[ 1.10826308e-01  1.34328023e-01  1.25470802e-01  1.06860280e-01
  -8.55904435e-02  6.89996973e-02  7.93986592e-02  8.19585118e-02
  -9.01758557e-05  1.65726877e-03 -2.97042813e-02]] Obj:  0.08515439406524136
LR:  0.1 S:  20 lamda:  0.1 55
w:  [[ 1.09760181e-01  1.33039087e-01  1.24299719e-01  1.05683051e-01
  -8.47434789e-02  6.83365528e-02  7.87188846e-02  8.12001407e-02
   1.10679488e-05  1.463

...

w:  [[-5.65258506e-04  2.02165317e-04  4.01972085e-04  1.36204097e-03
  -2.84223947e-04  8.78376466e-04 -1.03540644e-03 -3.20201612e-06
   2.37079545e-06 -1.89752835e-03 -1.65694198e-03]] Obj:  0.634791646358166
LR:  0.1 S:  20 lamda:  10.0 143
w:  [[-0.00132957 -0.00049135 -0.00027743  0.00039559 -0.00021835  0.00089335
  -0.0026337  -0.00102544 -0.00300412 -0.00107998 -0.00299672]] Obj:  2.197030470834358
LR:  0.1 S:  20 lamda:  10.0 144
w:  [[-1.14063450e-17  0.00000000e+00  4.71343284e-18  1.24709150e-02
   0.00000000e+00  1.14520683e-02 -7.05954044e-18  0.00000000e+00
   0.00000000e+00 -4.45394048e-18  0.00000000e+00]] Obj:  0.7413713354025145
LR:  0.1 S:  20 lamda:  10.0 145
w:  [[ 0.004429    0.0020223   0.00118916  0.00547851 -0.00103775  0.00441717
  -0.0035312   0.00129024 -0.00389047  0.00212618 -0.00350848]] Obj:  0.9749102432931113
LR:  0.1 S:  20 lamda:  10.0 146
w:  [[ 0.00726567  0.00692525  0.00793786  0.00136851 -0.00679317 -0.00345293
   0.01323653  0.00863112  0.009

  -1.92294334e+10 -1.90993191e+10 -3.09397691e+10]] Obj:  2.311357703075362e+23
LR:  0.1 S:  20 lamda:  100.0 14
w:  [[ 1.60241782e+11  1.63258620e+11  1.69442024e+11  9.24508369e+10
  -8.43170014e+10 -1.62984922e+10  3.09737935e+11  2.00532884e+11
   1.79071262e+11  1.74446384e+11  2.84471171e+11]] Obj:  2.0366068858253328e+25
LR:  0.1 S:  20 lamda:  100.0 15
w:  [[-1.44536131e+12 -1.47303977e+12 -1.52927742e+12 -8.29705470e+11
   7.60055806e+11  1.45849640e+11 -2.79203742e+12 -1.80664655e+12
  -1.61898695e+12 -1.56119451e+12 -2.55825819e+12]] Obj:  1.6294366751219765e+27
LR:  0.1 S:  20 lamda:  100.0 16
w:  [[ 1.30478360e+13  1.33793376e+13  1.38855824e+13  7.46734923e+12
  -6.92962035e+12 -1.31264676e+12  2.52081194e+13  1.63219473e+13
   1.46889652e+13  1.39238606e+13  2.30264768e+13]] Obj:  1.3169371768085343e+29
LR:  0.1 S:  20 lamda:  100.0 17
w:  [[-1.17468346e+14 -1.20530567e+14 -1.25063309e+14 -6.73775462e+13
   6.24645217e+13  1.16845350e+13 -2.26858163e+14 -1.46851918e+14
 

w:  [[ 4.46502967e+68  4.58779328e+68  4.66590011e+68  2.44666043e+68
  -2.97873244e+68  1.30552597e+67  7.19359697e+68  5.08504410e+68
   3.98838063e+68  3.16104008e+68  6.02945162e+68]] Obj:  1.2135574357919968e+140
LR:  0.1 S:  20 lamda:  100.0 75
w:  [[-4.05859649e+69 -4.19858280e+69 -4.26770871e+69 -2.19090328e+69
   2.72318092e+69 -1.21195598e+68 -6.57560958e+69 -4.62471042e+69
  -3.62287821e+69 -2.83956824e+69 -5.47288770e+69]] Obj:  9.897477978552625e+141
LR:  0.1 S:  20 lamda:  100.0 76
w:  [[ 3.69264950e+70  3.82299021e+70  3.88906041e+70  1.98798033e+70
  -2.48357077e+70  9.47879374e+68  6.00332413e+70  4.20730881e+70
   3.33494975e+70  2.58383558e+70  5.00581988e+70]] Obj:  8.032083935496006e+143
LR:  0.1 S:  20 lamda:  100.0 77
w:  [[-3.33147593e+71 -3.45291290e+71 -3.50968481e+71 -1.79171036e+71
   2.22776016e+71 -6.13265979e+69 -5.43985182e+71 -3.79611844e+71
  -3.01643618e+71 -2.35901371e+71 -4.54446304e+71]] Obj:  6.576622219536883e+145
LR:  0.1 S:  20 lamda:  100.0 78

w:  [[ 7.69087937e+131  8.20490080e+131  8.16380735e+131  4.36818148e+131
  -5.81108354e+131  1.32048918e+131  1.03228446e+132  8.06246917e+131
   4.98576038e+131  2.95965228e+131  7.13790794e+131]] Obj:  3.0559013271937525e+266
LR:  0.1 S:  20 lamda:  100.0 141
w:  [[-6.92903669e+132 -7.40577421e+132 -7.37105109e+132 -3.94517025e+132
   5.22379593e+132 -1.20156890e+132 -9.30248698e+132 -7.26746722e+132
  -4.49719326e+132 -2.62136059e+132 -6.40759903e+132]] Obj:  2.286243755178056e+268
LR:  0.1 S:  20 lamda:  100.0 142
w:  [[ 6.24203363e+133  6.66308643e+133  6.62974988e+133  3.53643518e+133
  -4.69844939e+133  1.07224283e+133  8.38304666e+133  6.54075393e+133
   4.04744919e+133  2.37903241e+133  5.78413557e+133]] Obj:  1.7781491439211633e+270
LR:  0.1 S:  20 lamda:  100.0 143
w:  [[-5.65335656e+134 -6.00990669e+134 -5.97418773e+134 -3.17222157e+134
   4.22277019e+134 -9.41148273e+133 -7.61511476e+134 -5.91407833e+134
  -3.72297469e+134 -2.16998646e+134 -5.28579463e+134]] Obj:  1.54386

C:\Users\ZHOUQI~1\AppData\Local\Temp/ipykernel_19584/2502419380.py:5: RuntimeWarning: overflow encountered in double_scalars
  regul = (lamda/2)* np.sum(np.power(w, 2)) # The second term in formula
C:\Users\ZHOUQI~1\AppData\Local\Temp/ipykernel_19584/2502419380.py:3: RuntimeWarning: overflow encountered in power
  inner = np.power(y - ((X * w.T)), 2) #RMSE
C:\Users\ZHOUQI~1\AppData\Local\Temp/ipykernel_19584/2502419380.py:5: RuntimeWarning: overflow encountered in power
  regul = (lamda/2)* np.sum(np.power(w, 2)) # The second term in formula


In [21]:
# The best parameter is LR:  0.1 S:  20 lamda:  0.01 at 189 times irters
w_best = batch_gradientDescent(XTrain, yTrain, w, S[0], eta[1], gamma[0], 189)

w:  [[ 0.00643977  0.00673664  0.00696254  0.00354669 -0.00315053 -0.00094174
   0.01305254  0.00817902  0.00762773  0.00721832  0.01199153]] Obj:  0.640957482682971
LR:  0.1 S:  20 lamda:  0.01 1
w:  [[ 0.00658114  0.00674891  0.00702833  0.00375065 -0.00343688 -0.00024357
   0.01289834  0.00831476  0.00726267  0.00755198  0.01176564]] Obj:  0.9203073569882795
LR:  0.1 S:  20 lamda:  0.01 2
w:  [[ 0.01317433  0.01095564  0.01268326  0.01436999 -0.01335358  0.01396122
   0.00955486  0.01797578 -0.00054778  0.00689276  0.00256469]] Obj:  0.5065320315673546
LR:  0.1 S:  20 lamda:  0.01 3
w:  [[ 0.02396963  0.02260975  0.02512929  0.04322486 -0.02659364  0.01533331
   0.01028027  0.02928463 -0.01451235  0.0305563  -0.00123439]] Obj:  0.8613563799553101
LR:  0.1 S:  20 lamda:  0.01 4
w:  [[ 0.02933189  0.02571953  0.02759039  0.04725044 -0.02588754  0.02354704
   0.01142404  0.03090458 -0.01662075  0.02598602 -0.00900724]] Obj:  0.5936641571562179
LR:  0.1 S:  20 lamda:  0.01 5
w:  [[ 0.03

w:  [[ 0.12119849  0.14713728  0.13752086  0.13059066 -0.09140207  0.07641535
   0.0844063   0.08195086 -0.0015774   0.00669652 -0.03556714]] Obj:  0.1512764552838527
LR:  0.1 S:  20 lamda:  0.01 57
w:  [[ 0.12313607  0.14927015  0.13971152  0.13270984 -0.09176779  0.08061053
   0.08498085  0.08333509 -0.00174955  0.00326115 -0.03884461]] Obj:  0.16794872077926104
LR:  0.1 S:  20 lamda:  0.01 58
w:  [[ 0.12439434  0.15072446  0.14137001  0.13286559 -0.09229574  0.07945809
   0.0876603   0.08330884  0.00115435  0.00285573 -0.03632966]] Obj:  0.06955061469631892
LR:  0.1 S:  20 lamda:  0.01 59
w:  [[ 0.12517796  0.15181051  0.14243858  0.13319241 -0.09186602  0.07907744
   0.08912677  0.08446992  0.00051278  0.00401265 -0.03599555]] Obj:  0.04507946319926304
LR:  0.1 S:  20 lamda:  0.01 60
w:  [[ 0.12505301  0.15167827  0.14220796  0.13347802 -0.09205274  0.07923914
   0.08874193  0.08464265  0.00079312  0.00398488 -0.03562796]] Obj:  0.1847242715812605
LR:  0.1 S:  20 lamda:  0.01 61
w:

w:  [[ 0.16812225  0.19629991  0.17544522  0.15847074 -0.0800485   0.09340709
   0.11456177  0.11537299 -0.00496484  0.00875515 -0.03604981]] Obj:  0.17108856773092346
LR:  0.1 S:  20 lamda:  0.01 120
w:  [[ 0.16272233  0.19367387  0.17250401  0.15989875 -0.07815844  0.09333747
   0.10966457  0.11322665 -0.00780994  0.01047967 -0.03837413]] Obj:  0.07581080744118984
LR:  0.1 S:  20 lamda:  0.01 121
w:  [[ 0.16802892  0.20317098  0.18027137  0.16105728 -0.08245503  0.09481046
   0.11981727  0.11926261 -0.00452216  0.00939596 -0.03421339]] Obj:  0.11473465426457842
LR:  0.1 S:  20 lamda:  0.01 122
w:  [[ 0.16642888  0.20141867  0.17848722  0.15898869 -0.08083106  0.09461193
   0.11813791  0.11829386 -0.00465169  0.00726586 -0.03630709]] Obj:  0.03973015476810391
LR:  0.1 S:  20 lamda:  0.01 123
w:  [[ 0.15693988  0.19243417  0.16971315  0.15330599 -0.07183987  0.08944175
   0.11435748  0.11151916 -0.00714869  0.01292938 -0.03380427]] Obj:  0.06971500893708944
LR:  0.1 S:  20 lamda:  0.01

w:  [[ 0.17629414  0.22260068  0.17551402  0.16075907  0.0147956   0.11055755
   0.14919054  0.08445567  0.00840327 -0.03126769 -0.04364038]] Obj:  0.13983051419011933
LR:  0.1 S:  20 lamda:  0.01 186
w:  [[ 0.17752006  0.22308523  0.17597975  0.16135862  0.01328464  0.11105287
   0.14990119  0.0855061   0.00786008 -0.02918188 -0.04209999]] Obj:  0.12145553140318568
LR:  0.1 S:  20 lamda:  0.01 187
w:  [[ 0.1769097   0.22258968  0.17548917  0.16092297  0.01391728  0.11055423
   0.14967812  0.08502902  0.00813277 -0.02947579 -0.04205994]] Obj:  0.23207175511776723
LR:  0.1 S:  20 lamda:  0.01 188
w:  [[ 0.17340647  0.2183891   0.1712314   0.16230374  0.01479166  0.10899274
   0.14575205  0.08327597  0.00370465 -0.02407273 -0.04094432]] Obj:  0.048718935807551586
LR:  0.1 S:  20 lamda:  0.01 189


In [22]:
# we can get the optimized w with the best parameters we selected as the objective function is minimized
w_best

matrix([[ 0.17340647,  0.2183891 ,  0.1712314 ,  0.16230374,  0.01479166,
          0.10899274,  0.14575205,  0.08327597,  0.00370465, -0.02407273,
         -0.04094432]])

## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [23]:
# Write your code here
## combine data
ori_train = data_training_unproc
ori_val = data_val_unproc 
combine = [ori_train, ori_val]

combine_data_unproc = pd.concat(combine) # 6523 85%
combine_df = combine_data_unproc

combine_df.replace(to_replace = -200, value =np.nan,inplace=True) # We should not calcaluate the mean with -200 for na value
## Standardisation and clean
clean_combine_df = combine_df.fillna(combine_df.mean())
stand_combine_df = (clean_combine_df-clean_combine_df.mean())/clean_combine_df.std()
stand_combine_df

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6459,0.646235,0.326567,-0.114520,0.036463,1.179855,-0.543353,0.186359,-0.125778,0.032424,-0.452062,1.621243,0.554943
4266,0.098052,0.325401,0.408770,0.557150,0.000000,-0.748033,0.000000,0.563788,0.067625,2.375559,-1.619377,1.117220
5989,-0.998315,-1.605140,-1.133507,-1.383856,-0.360853,1.874677,-0.448388,-1.625055,-1.349195,-0.829655,-0.709601,-1.321748
3081,-0.655700,-0.928984,-0.497213,-0.404193,0.000000,0.153963,0.000000,0.134175,-0.074434,0.395359,-0.196845,0.374502
217,1.605556,1.668386,1.586200,1.533920,-0.029008,-0.591513,0.609524,1.612336,1.072096,-0.258942,0.087369,-0.266737
...,...,...,...,...,...,...,...,...,...,...,...,...
1471,-1.203884,-1.295041,-1.034270,-1.192938,-1.048246,1.454281,-1.315876,-0.236458,-0.870217,-0.310825,0.622101,0.065353
8610,0.783281,1.021375,0.639930,0.766389,1.609831,-1.155385,1.538371,-0.127962,1.566789,-1.045834,0.494644,-0.871502
8396,0.235098,0.183175,-0.015907,0.140600,0.560727,-0.784152,2.471449,-0.579420,0.190827,-1.385955,1.366330,-0.816681
8138,2.427831,3.425225,2.284934,2.035322,3.292285,-1.750361,3.161207,0.782963,3.472644,-0.725889,0.368652,-0.616346


In [24]:
newytrain = np.matrix(np.array(stand_combine_df.iloc[:,:1]))
newXtrain = np.matrix(np.array(stand_combine_df.iloc[:,1:]))

### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [25]:
# Write your code here
test_data = data_test_unproc
test_data.replace(to_replace = -200, value =np.nan,inplace=True)
## Standardisation and clean
clean_test_df = test_data.fillna(test_data.mean())
stand_test_df = (clean_test_df-clean_test_df.mean())/clean_test_df.std()
stand_test_df

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
133,-0.584556,3.085904e-01,-6.997705e-01,-0.686784,-7.276813e-01,1.234756,-0.950148,4.099754e-02,-0.272173,-0.816056,1.044403e+00,-0.319754
528,1.459859,2.001555e+00,1.453429e+00,1.451874,-1.695802e-01,-0.531015,0.433867,1.695947e+00,0.925791,0.556116,-9.673365e-01,-0.291804
8475,0.966379,-9.066008e-02,-3.040970e-01,-0.179532,1.318940e-01,-0.482231,1.081229,-1.022267e+00,-0.027468,-1.252263,4.731684e-01,-1.071651
1392,1.530356,4.685276e-01,9.413117e-01,1.038063,-1.147667e-01,-0.595103,0.456190,1.350368e+00,0.231931,-0.619619,1.275234e+00,0.089948
1610,0.190912,6.249106e-01,2.694595e-02,0.185651,-5.781900e-01,-0.022136,0.121347,5.759456e-01,-0.059414,0.651446,-5.378148e-01,0.370228
...,...,...,...,...,...,...,...,...,...,...,...,...
357,-0.161574,-1.048767e-01,-2.338895e-01,-0.098487,-5.981222e-01,0.563266,-0.459046,1.419450e-01,-0.640187,-0.232522,-4.297038e-01,-0.640485
7394,2.446818,1.990892e+00,2.293244e+00,2.066869,2.346359e+00,-1.571729,1.197307,1.267104e+00,1.874578,-0.573398,8.340250e-01,-0.132897
8071,-0.232071,-1.077496e-15,-2.373181e-16,0.000000,-6.085866e-01,0.000000,-0.142062,-6.701549e-16,0.000000,0.000000,-4.152293e-16,0.000000
4283,-0.866544,-9.175944e-01,-7.941023e-01,-0.825039,1.416267e-16,0.093605,0.000000,-3.377398e-01,-0.998618,0.870994,-6.473866e-01,0.563570


In [26]:
# we used the best parameters to train a new regularized model
new_w = batch_gradientDescent(newXtrain, newytrain, w_best, S[1], eta[0], gamma[1], 200)

w:  [[ 0.17326799  0.21817917  0.17106157  0.16214143  0.01477759  0.10888375
   0.14561503  0.08323176  0.00361857 -0.02396419 -0.04089643]] Obj:  0.1032374372872807
LR:  0.01 S:  73 lamda:  0.1 1
w:  [[ 0.17380036  0.21899072  0.17166666  0.16262313  0.01433964  0.10896123
   0.14613876  0.08378434  0.00371436 -0.02378531 -0.04053703]] Obj:  0.1655321410224444
LR:  0.01 S:  73 lamda:  0.1 2
w:  [[ 0.17360312  0.2187454   0.17146295  0.16244102  0.01436243  0.10884603
   0.14600417  0.08365866  0.00374159 -0.02378249 -0.04047325]] Obj:  0.10060441878734547
LR:  0.01 S:  73 lamda:  0.1 3
w:  [[ 0.1736913   0.21883395  0.17156676  0.16257635  0.01414693  0.10887805
   0.14613429  0.08369692  0.00377147 -0.02348923 -0.04011828]] Obj:  0.09472927580320661
LR:  0.01 S:  73 lamda:  0.1 4
w:  [[ 0.17430723  0.21915144  0.17193155  0.16250738  0.01382008  0.10897727
   0.14667776  0.08394832  0.00359167 -0.02279463 -0.03983517]] Obj:  0.2502274914797768
LR:  0.01 S:  73 lamda:  0.1 5
w:  [[ 0

w:  [[ 0.16185388  0.20428324  0.16014106  0.15080159  0.0130291   0.10028838
   0.13638318  0.07800647  0.00252041 -0.0209671  -0.03736785]] Obj:  0.0998478146775118
LR:  0.01 S:  73 lamda:  0.1 80
w:  [[ 0.1617146   0.2041218   0.16002052  0.15065551  0.01299462  0.1002296
   0.13628763  0.07795508  0.00256847 -0.02098451 -0.03731566]] Obj:  0.09589453464874949
LR:  0.01 S:  73 lamda:  0.1 81
w:  [[ 0.16154073  0.20391125  0.15985495  0.15050715  0.01298552  0.10012808
   0.13613568  0.07787022  0.00256517 -0.02098067 -0.0372938 ]] Obj:  0.21390575139040208
LR:  0.01 S:  73 lamda:  0.1 82
w:  [[ 0.16137421  0.20370418  0.1596916   0.15035088  0.01297727  0.10002376
   0.13599893  0.07778704  0.00256532 -0.02096311 -0.03725539]] Obj:  0.11651491595948413
LR:  0.01 S:  73 lamda:  0.1 83
w:  [[ 0.16124704  0.20352354  0.15955149  0.1502405   0.01293593  0.09996079
   0.13585746  0.07775281  0.00246416 -0.02086623 -0.03724119]] Obj:  0.18318014733320775
LR:  0.01 S:  73 lamda:  0.1 84
w:

w:  [[ 0.15492328  0.19565499  0.15364185  0.14423649  0.00984491  0.09507094
   0.12978652  0.07609665  0.00145715 -0.01894983 -0.03514002]] Obj:  0.09223554363787996
LR:  0.01 S:  73 lamda:  0.1 147
w:  [[ 0.15484272  0.19552423  0.15355913  0.1441539   0.00973869  0.09505932
   0.12969471  0.07609282  0.00140751 -0.01889004 -0.03513027]] Obj:  0.21972651580817384
LR:  0.01 S:  73 lamda:  0.1 148
w:  [[ 0.15471059  0.19534791  0.15342376  0.14403773  0.00972089  0.0949992
   0.12955873  0.07604109  0.00135957 -0.01884642 -0.0351391 ]] Obj:  0.0902177995490696
LR:  0.01 S:  73 lamda:  0.1 149
w:  [[ 0.15456957  0.19518863  0.15329661  0.14384549  0.00974295  0.09491371
   0.1295002   0.07596681  0.00147749 -0.01898066 -0.03504802]] Obj:  0.18511845939521104
LR:  0.01 S:  73 lamda:  0.1 150
w:  [[ 0.15441291  0.19498115  0.15312701  0.14369051  0.00977559  0.09480088
   0.12936342  0.07587555  0.00146864 -0.01895774 -0.03501909]] Obj:  0.12312620645097094
LR:  0.01 S:  73 lamda:  0.1 1

In [27]:
# prepare the test data to cumpute the RMSE
testytrain = np.matrix(np.array(stand_test_df.iloc[:,:1]))
testXtrain = np.matrix(np.array(stand_test_df.iloc[:,1:]))
testytrain.shape,testXtrain.shape

((1151, 1), (1151, 11))

In [28]:
# The function RMSE to mearsure the error
def RMSE(X, y, w):
    inner = np.power(((X * w.T) - y), 2)# (m,n) @ (n, 1) -> (n, 1)
#     return np.sum(inner) / (2 * len(X))
    return np.sum(inner) / (2 * X.shape[0])

In [29]:
# The RMSE we got is small over test data which means the model was sucessful trained
RMSE(testXtrain,testytrain, new_w)

0.07957528713078985

# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [30]:
# Write your code here
from sklearn.impute import SimpleImputer
q2_df =  data_training_unproc.copy() # Unprocess data`a
q2_df.replace(to_replace = -200, value =np.nan,inplace=True) # We should not calcaluate the mean with -200 for na value
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') # relace with mean value
imp_mean.fit(q2_df)
q2_data_clean = imp_mean.transform(q2_df) # clean data

In [31]:
from sklearn.preprocessing import StandardScaler # standard
ss = StandardScaler()
ss.fit(q2_data_clean)
data_ss = ss.transform(q2_data_clean) ## data after process

In [32]:
from sklearn.pipeline import Pipeline
pre_proc = [("FillingNa",SimpleImputer(missing_values=np.nan, strategy='mean')), ("standarlize", StandardScaler())]
pipe = Pipeline(pre_proc)
pipe.fit(q2_df)
pipe.transform(q2_df)
data_after_proc = pipe.transform(q2_df) # data processing using pipeline
data_after_proc, data_ss # same as we did before without pipeline

(array([[ 6.55274681e-01,  3.24477790e-01, -1.16451498e-01, ...,
         -4.39574001e-01,  1.61662138e+00,  5.67584863e-01],
        [ 1.00726127e-01,  3.23305224e-01,  4.11221597e-01, ...,
          2.39086215e+00, -1.62431465e+00,  1.13100293e+00],
        [-1.00837098e+00, -1.61846473e+00, -1.14397377e+00, ...,
         -8.17542525e-01, -7.14449681e-01, -1.31291499e+00],
        ...,
        [-1.76548151e-01, -1.08199211e-01,  2.15457734e-02, ...,
          2.41105886e+00, -1.57010371e+00,  1.32412186e+00],
        [-1.00837098e+00, -1.06644293e-15,  0.00000000e+00, ...,
          0.00000000e+00, -4.16423781e-16,  0.00000000e+00],
        [-8.00415274e-01, -6.44062055e-01, -6.08234130e-01, ...,
          3.13477807e-01,  1.25472657e+00,  1.74379052e+00]]),
 array([[ 6.55274681e-01,  3.24477790e-01, -1.16451498e-01, ...,
         -4.39574001e-01,  1.61662138e+00,  5.67584863e-01],
        [ 1.00726127e-01,  3.23305224e-01,  4.11221597e-01, ...,
          2.39086215e+00, -1.62431465e

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [33]:
# Write your code here
Training_data = air_quality.iloc[index[0:N], :].copy()  #70% 5372 * 12
pipe.fit(Training_data)
Train_after_proc = pipe.transform(Training_data)

Val_data = air_quality.iloc[index[N:N+Nval], :].copy() #15% 1151 * 12
pipe.fit(Val_data)
Val_after_proc = pipe.transform(Val_data)
Val_after_proc, Train_after_proc

(array([[ 0.54269294, -3.64264182, -4.66972092, ..., -4.65268312,
         -4.49032786, -4.74211649],
        [-0.30449688,  0.01280512,  0.15307029, ...,  0.23540783,
          0.12936016,  0.21090082],
        [ 0.60786139,  0.80224778,  0.31098501, ...,  0.18920447,
          0.54461829,  0.21862863],
        ...,
        [ 0.2168507 ,  0.28783675,  0.20572012, ..., -0.07242891,
          0.64642962,  0.20245411],
        [ 2.30224104,  2.31128379,  0.5923478 , ...,  0.05504778,
          0.32544119,  0.20438828],
        [-0.89101292, -0.8108346 ,  0.04911301, ...,  0.3918312 ,
         -0.17230298,  0.21044435]]),
 array([[ 0.65527468,  0.36855219,  0.19021649, ...,  0.12399278,
          0.71735627,  0.21787512],
        [ 0.10072613,  0.36782004,  0.27766366, ...,  0.66807832,
         -0.31713352,  0.2232673 ],
        [-1.00837098, -0.84462584,  0.01993319, ...,  0.05133711,
         -0.02670939,  0.19987783],
        ...,
        [-0.17654815,  0.09838762,  0.2130857 , ...,  

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [35]:
whole_train_set = np.vstack((data_after_proc, Val_after_proc)) # 6523 * 12 85%
data_after_proc.shape # 70%

(5372, 12)

In [36]:
# Write your code here
## PredefinedSplit to  aasign the training and validation data in cross validator
from sklearn.model_selection import PredefinedSplit
# make the test fold
test_fold = np.zeros((np.shape(whole_train_set)[0], 1)) # 0 assign ti val
test_fold[0:np.shape(data_after_proc)[0]] = -1 # train set for -1 for 5372 * -1
ps = PredefinedSplit(test_fold) # PS to tell which part used for val and which for train
test_fold.shape

(6523, 1)

In [37]:
whole_train_set_label = np.matrix(whole_train_set)[ : ,0] # We should get the label(CO) of train_set and val_set
train_set = np.asarray(whole_train_set)[ : ,1:] # (6523, 11) 70%
train_set_label = np.matrix(Train_after_proc)[ : ,0]#(5372, 1)
val_set_label =  np.matrix(Val_after_proc)[ : ,0]# 1151, 1
train_set_label.shape
whole_train_set.shape

(6523, 12)

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# set the grid of parameter
max_features =  np.linspace(1, 12, num=12, dtype = int)
n_estimators =  np.linspace(20, 100, num=5, dtype = int)# the number of trees you want to build before taking the maximum voting or averages of predictions
max_samples = np.linspace(1.0, 100, num=5, dtype = int) # the grid of maximum number selected

param_grid = dict(n_estimators = n_estimators, max_samples = max_samples, max_features = max_features ) # the para_grid used with regression to find best value
grid_regression = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=ps, scoring='neg_mean_squared_error') # the model

grid_regression.fit(whole_train_set,np.ravel( whole_train_set_label)) # we train our model with whole train data set


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
                         'max_samples': array([  1,  25,  50,  75, 100]),
                         'n_estimators': array([ 20,  40,  60,  80, 100])},
             scoring='neg_mean_squared_error')

In [39]:
from sklearn.metrics import mean_squared_error
## Training a Random Forest using the best value for the  n_estimators and max_samples
regr = RandomForestRegressor(n_estimators=grid_regression.best_params_["n_estimators"],
                             max_samples=grid_regression.best_params_["max_samples"],
                            max_features=grid_regression.best_params_["max_features"])
regr.fit(Train_after_proc,np.ravel(np.asarray(train_set_label))) # attribute + labels 70%
predictions = regr.predict(Val_after_proc) # The prediction of the models, use Validation set

## Computing the RMSE for the validation dataset and the best parameter
error_mod = np.sqrt(mean_squared_error(np.asarray(val_set_label), predictions)) # Compute the error useing RMSE

print('The RMSE on the validation data is :',error_mod)
print("Max of features: ", grid_regression.best_params_["max_features"])
print("Number of estimator: ", grid_regression.best_params_["n_estimators"])
print("Max samples: ", grid_regression.best_params_["max_samples"])

The RMSE on the validation data is : 0.11402900112664492
Max of features:  12
Number of estimator:  100
Max samples:  100


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [40]:
# Write your code here
best_n_est = grid_regression.best_params_["n_estimators"]
best_n_sam = grid_regression.best_params_["max_samples"]
best_n_feat = grid_regression.best_params_["max_features"]
# The best parameter we get

In [41]:
whole_data = np.vstack((data_training_unproc, data_val_unproc) ) # Combine the train and val dataset
test_data = np.asarray( data_test_unproc)

In [42]:
pre_proc = [("FillingNa",SimpleImputer(missing_values=np.nan, strategy='mean')), ("standarlize", StandardScaler())]
pipe = Pipeline(pre_proc)
pipe.fit(whole_data)
whole_data_after_proc = pipe.transform(whole_data)

pipe.fit(whole_data)
test_data_after_proc = pipe.transform(test_data)
test_label = np.matrix(test_data_after_proc)[ : ,0]
whole_train_label = np.matrix(whole_data_after_proc)[ : ,0]

In [43]:
## Training a Random Forest using the best value for the  n_estimators and max_samples
regr = RandomForestRegressor(n_estimators = best_n_est, max_features = best_n_feat, max_samples = best_n_sam )
regr.fit(whole_data_after_proc , np.ravel(np.asarray(whole_train_label))) # attribute and labels # 85 % data in train
predictions = regr.predict(test_data_after_proc) # 15% test data

## Computing the RMSE for the validation dataset and the best parameter
error_mod = np.sqrt(mean_squared_error(np.asarray(test_label), predictions))
print('The RMSE over the test data is :',error_mod)

The RMSE over the test data is : 0.14161445884768223
